In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import gc
!pip install segmentation-models
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
trainImgPath = "/kaggle/input/severstal-steel-defect-detection/train_images/"
trainCsv = "/kaggle/input/severstal-steel-defect-detection/train.csv"
data=pd.read_csv(trainCsv)
data.ClassId=data.ClassId.astype(int)

In [ ]:
WIDTH=192
HEIGHT=192
TRAINING_SIZE=7095

In [ ]:
def convert_to_mask(encoded_pixels):
    counts=[]
    mask=np.zeros((256*1600), dtype=np.uint8) #don't change this
    pre_mask=np.asarray([int(point) for point in encoded_pixels.split()])
    for index,count in enumerate(pre_mask):
        if(index%2!=0):
            counts.append(count)
    i=0
    for index,pixel in enumerate(pre_mask):
        if(index%2==0):
            if(i==len(counts)):
                break
            mask[pixel:pixel+counts[i]]=1
            i+=1
    mask=np.reshape(mask,(1600,256)) #don't change this
    mask=cv2.resize(mask,(HEIGHT,WIDTH)).T
    return mask

In [ ]:
def generate_imgs():
    list_of_imgs=np.empty((TRAINING_SIZE,HEIGHT,WIDTH,3), dtype=np.uint8)
    list_of_masks=np.empty((TRAINING_SIZE,HEIGHT,WIDTH,4), dtype=np.uint8)
    i=0
    encoder=np.zeros(4)
    for subdir, dirs, files in os.walk(trainImgPath):
        for file in files:
            imgpath = subdir + os.sep + file
            row=data[data.ImageId==file]
            if(row.empty):
                continue
            img_class=row.ClassId
            encoder[img_class-1]=1
            rle=row.EncodedPixels.to_list()
            rle=''.join(rle)
            mask=convert_to_mask(rle)
            y=np.zeros((WIDTH,HEIGHT,4))
            y[...,0]=mask
            y[0,...]=encoder
            encoder[img_class-1]=0
            list_of_masks[i]=y
            img=cv2.imread(imgpath)
            img=cv2.resize(img,(HEIGHT,WIDTH))
            list_of_imgs[i]=img
            i+=1
            del y
            del mask
            del img
            gc.collect()
    return list_of_imgs,list_of_masks
    

In [ ]:
x_train,y_train=generate_imgs()

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x_train,y_train,stratify=[1,2,3,4])

In [ ]:
print(x_train.shape,y_train.shape)

In [ ]:
from keras import backend as K
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
 
network = 'resnet34'
process_input = get_preprocessing(network)
x_train = process_input(x_train)
model = Unet(network,input_shape = (192, 192, 3),classes=4,activation='sigmoid')
model.compile('adam', loss='binary_crossentropy',metrics=[dice_coef])


In [ ]:
model.fit(
    x=x_train,
    y=y_train,
    batch_size=16,
    epochs=10, 
)